In [2]:
import json
from web3 import Web3
import os
import platform
from getpass import getpass
import random

In [3]:
operating_system = platform.system()
d = '/'
if operating_system == 'Windows':
    d = '\\'

ganache_url = "HTTP://127.0.0.1:7545"
web3 = Web3(Web3.HTTPProvider(ganache_url))

# This is our main account that controls everything
web3.eth.defaultAccount = web3.eth.accounts[0]

In [4]:
def getContract(filename):
    with open('..'+ d +'build'+ d +'contracts'+ d +filename) as contractFile:
        contractJson=json.load(contractFile)
        contractABI =contractJson['abi']
        contractAdd=web3.toChecksumAddress(contractJson['networks']['5777']['address']) 
        return contractABI, contractAdd



# Get the organization contract address
organizationContractABI, organizationContractAdd = getContract('Organization.json')	
organizationContract = web3.eth.contract(address=organizationContractAdd, abi=organizationContractABI)

#get the user contract address
userContractABI,userContractAdd = getContract('User.json')
userContract = web3.eth.contract(address=userContractAdd, abi=userContractABI)

# Get the accounts contract address
accountsContractABI,accountsContractAdd = getContract('Accounts.json')
accountsConract = web3.eth.contract(address=accountsContractAdd, abi=accountsContractABI)

loansContractABI, loansContractAddress= getContract('Loans.json')
loansContract = web3.eth.contract(address=loansContractAddress, abi=loansContractABI)

In [5]:
# This should be changed to be a more robus way. We can give the loansContractAddress using the constructor
# but this has problems when doing this in the 2_deploy_contracts.js
userContract.functions.setLoansContractAddress(loansContractAddress).transact()
organizationContract.functions.setLoansContractAddress(loansContractAddress).transact()

HexBytes('0x504f64390b71e0a4dbbf8b167c8cfc4f1a5a40df061c55c4cdedaa027e1d6ea9')

In [6]:
accountsConract.functions.add(web3.eth.accounts[1], False).transact()
accountsConract.functions.add(web3.eth.accounts[2], False).transact()
accountsConract.functions.add(web3.eth.accounts[5], True).transact()
accountsConract.functions.add(web3.eth.accounts[6], True).transact()

HexBytes('0xafe464ffa2d54ce03b36f38e991bb24406787bfcb24b97f03671e5fd82fdd1f2')

In [7]:
def deleteUser():
    accountsConract.functions.deleteAccount(web3.eth.accounts[1]).transact()

def createLoan(_loanie, _loaner, _amount, _organizationContract, _installmentsNum ,  _interest):
    
    transaction = _organizationContract.functions.createLoan(_loanie, _amount, _installmentsNum ,  _interest
    ).buildTransaction({
    'gas': 3000000,
    'gasPrice': web3.toWei('1', 'gwei'),
    'from': _loaner,
    'nonce': web3.eth.getTransactionCount(_loaner)
    }) 
    _privateKey = getpass("Enter the password of the organization: ")
    signed_txn = web3.eth.account.signTransaction(transaction, private_key=_privateKey)
    transaction_hash = web3.eth.sendRawTransaction(web3.toHex(signed_txn.rawTransaction))
    
    # _organizationContract.functions.createLoan(_loanie, _loaner, _amount).transact()
    return True

In [99]:
deleteUser()

In [8]:
def getPendingLoans(_userContract, _loanieAddress, _privateKey):
    transaction = _userContract.functions.getPendingLoans(
    ).buildTransaction({
    'gas': 300000,
    'gasPrice': web3.toWei('1', 'gwei'),
    'from': _loanieAddress,
    'nonce': web3.eth.getTransactionCount(_loanieAddress)
    }) 
    
    signed_txn = web3.eth.account.signTransaction(transaction, private_key=_privateKey)
    transaction_hash = web3.eth.sendRawTransaction(web3.toHex(signed_txn.rawTransaction))
    receipt = web3.eth.getTransactionReceipt(transaction_hash)
    rich_logs = _userContract.events.getAmounts().processReceipt(receipt)
    event_values = rich_logs[0]['args'] # Dictionary
    return event_values


In [9]:
# Suppose bank wants to create a loan
#loaner = input("Enter the loaner addrses: ")
loaner = web3.eth.accounts[5]
loaner
### Check to see whether it is a user or organization address
loanerIndex = accountsConract.functions.getIndex(loaner).call()

In [10]:
#loanie = input("Enter the loanie address: ")
loanie = web3.eth.accounts[2]
loanie
loanieIndex = accountsConract.functions.getIndex(loanie).call()

## Get pending loans

In [17]:
# Get pending loans using 3 separate functions
def getPendingLoansList(_accountIndex, _userContract, _accountsContract, _loanieAddress):
    pendingLoans = [] # 'amount loanerAddress id'
    if _accountIndex != -1:
        loanieType = _accountsContract.functions.getType(_accountIndex).call()
        if not loanieType:
            privateKey = getpass("Enter your password: ")
            values = getPendingLoans(_userContract, _loanieAddress, privateKey)
            for i in range(len(values['_amounts'])):
                string = ''
                for key in values:
                    string += str(values[key][i]) + ' '
                pendingLoans.append(string)
    else:
        print("This account is not registered in our system.")
    return pendingLoans

In [18]:
getPendingLoansList(loanieIndex, userContract, accountsConract, loanie)

Enter your password: ········


[]

## Confirm or reject loans

In [16]:
# Confirm or reject loans
pendingLoans = getPendingLoansList(loanieIndex, userContract, accountsConract, loanie)
private_key = getpass('Enter your loanie password: ')
for pendingLoan in pendingLoans:
    print('for id: ' + pendingLoan.split(' ')[0])
    loanId = pendingLoan.split(' ')[2]
    choice = input('c/r?')
    if choice == 'c':
        transaction = userContract.functions.validateLoan(True,
        int(loanId)).buildTransaction({
        'gas': 1200000,
        'gasPrice': web3.toWei('1', 'gwei'),
        'from': loanie,
        'nonce': web3.eth.getTransactionCount(loanie)
        }) 
        signed_txn = web3.eth.account.signTransaction(transaction, private_key=private_key)
        transaction_hash = web3.eth.sendRawTransaction(web3.toHex(signed_txn.rawTransaction))
    elif choice == 'r':
        transaction = userContract.functions.validateLoan(False,
        int(loanId)).buildTransaction({
        'gas': 300000,
        'gasPrice': web3.toWei('1', 'gwei'),
        'from': loanie,
        'nonce': web3.eth.getTransactionCount(loanie)
        }) 
        signed_txn = web3.eth.account.signTransaction(transaction, private_key=private_key)
        transaction_hash = web3.eth.sendRawTransaction(web3.toHex(signed_txn.rawTransaction))
    else:
        print("Not recognized operation.")

Enter your password: ········
Enter your loanie password: ········
for id: 486176
c/r?c
for id: 546769
c/r?c
for id: 30101
c/r?r
for id: 377994
c/r?r


# Create a new loan

In [15]:
loaner
# Create a new loan
if loanerIndex != -1:
    loanerType = accountsConract.functions.getType(loanerIndex).call()
    if loanerType:
        # We will change this to inputs later
        amount=random.randint(1000,1000000)
        if amount%2!=0:
            amount+=1
        installmentsNum=random.randint(6,15)
        interest=random.randint(5,35)
        amount+=(amount*interest)//100
        if createLoan(_loanie=web3.eth.accounts[2], _loaner=loaner, _amount=amount, _organizationContract=organizationContract, _installmentsNum=installmentsNum, _interest=interest):
            print("Loan created")
else:
    print("This account is not registered in our system.")

Enter the password of the organization: ········
Loan created


## Get user loans

In [25]:
def getLoans(_userContract, _loanieAddress, _privateKey):
    transaction = _userContract.functions. getMyLoans(
    ).buildTransaction({
    'gas': 70000,
    'gasPrice': web3.toWei('1', 'gwei'),
    'from': _loanieAddress,
    'nonce': web3.eth.getTransactionCount(_loanieAddress)
    }) 
    
    signed_txn = web3.eth.account.signTransaction(transaction, private_key=_privateKey)
    transaction_hash = web3.eth.sendRawTransaction(web3.toHex(signed_txn.rawTransaction))
    receipt = web3.eth.getTransactionReceipt(transaction_hash)
    rich_logs = _userContract.events.getAmounts().processReceipt(receipt)
    event_values = rich_logs[0]['args'] # Dictionary
    return event_values

In [26]:
def getLoansList(_accountIndex, _userContract, _accountsContract, _loanieAddress):
    loans = [] # 'amount loanerAddress id'
    if _accountIndex != -1:
        loanieType = _accountsContract.functions.getType(_accountIndex).call()
        if not loanieType:
            privateKey = getpass("Enter your password: ")
            values = getLoans(_userContract, _loanieAddress, privateKey)
            for i in range(len(values['_amounts'])):
                string = ''
                for key in values:
                    string += str(values[key][i]) + ' '
                loans.append(string)
    else:
        print("This account is not registered in our system.")
    return loans

In [28]:
getLoansList(loanieIndex,userContract,accountsConract,loanie)

Enter your password: ········


['486176 0xeCcc469d6e5288422eC9a83C6D4eb57D60aBA6d7 1590157783 13 15 ',
 '546769 0xeCcc469d6e5288422eC9a83C6D4eb57D60aBA6d7 1590157789 9 19 ']

In [32]:
def getInstallments(_loanContract, _loanieAddress, _privateKey):
    myId = 1590157783
    transaction = _loanContract.functions.getMyInstallments(myId
    ).buildTransaction({
    'gas': 300000,
    'gasPrice': web3.toWei('1', 'gwei'),
    'from': _loanieAddress,
    'nonce': web3.eth.getTransactionCount(_loanieAddress)
    }) 
    
    signed_txn = web3.eth.account.signTransaction(transaction, private_key=_privateKey)
    transaction_hash = web3.eth.sendRawTransaction(web3.toHex(signed_txn.rawTransaction))
    receipt = web3.eth.getTransactionReceipt(transaction_hash)
    rich_logs = _loanContract.events.getLoanInstallments().processReceipt(receipt)
    event_values = rich_logs[0]['args'] # Dictionary
    return event_values

In [33]:
def getinstallmentsList(_accountIndex, _loanContract, _accountsContract, _loanieAddress):
    pendingLoans = [] # 'amount loanerAddress id'
    if _accountIndex != -1:
        loanieType = _accountsContract.functions.getType(_accountIndex).call()
        if not loanieType:
            privateKey = getpass("Enter your password: ")
            values = getInstallments(_loanContract, _loanieAddress, privateKey)
            for i in range(len(values['_amount'])):
                string = ''
                for key in values:
                    string += str(values[key][i]) + ' '
                pendingLoans.append(string)
    else:
        print("This account is not registered in our system.")
    return pendingLoans

In [34]:
getinstallmentsList(loanieIndex,loansContract,accountsConract,loanie)

Enter your password: ········


['37398 1592749823 0 False ',
 '37398 1595341823 0 False ',
 '37398 1597933823 0 False ',
 '37398 1600525823 0 False ',
 '37398 1603117823 0 False ',
 '37398 1605709823 0 False ',
 '37398 1608301823 0 False ',
 '37398 1610893823 0 False ',
 '37398 1613485823 0 False ',
 '37398 1616077823 0 False ',
 '37398 1618669823 0 False ',
 '37398 1621261823 0 False ',
 '37400 1623853823 0 False ']

In [55]:
# GetLoanerLoans

In [35]:
def getLoanerLoans(_organizationContract, _loanerAddress, _privateKey):
    transaction = _organizationContract.functions. getLoans( 
    ).buildTransaction({
    'gas': 70000,
    'gasPrice': web3.toWei('1', 'gwei'),
    'from': _loanerAddress,
    'nonce': web3.eth.getTransactionCount(_loanerAddress)
    }) 
    
    signed_txn = web3.eth.account.signTransaction(transaction, private_key=_privateKey)
    transaction_hash = web3.eth.sendRawTransaction(web3.toHex(signed_txn.rawTransaction))
    receipt = web3.eth.getTransactionReceipt(transaction_hash)
    rich_logs = _organizationContract.events.getLoanerLoans().processReceipt(receipt)
    event_values = rich_logs[0]['args'] # Dictionary
    return event_values


In [36]:
def getLoanerLoansList(_accountIndex,_organizationContract,_accountsContract, _loanerAddress):
    loans = [] # 'amount loanerAddress id'
    if _accountIndex != -1:
        loanieType = _accountsContract.functions.getType(_accountIndex).call()
        if loanieType:
            privateKey = getpass("Enter your password: ")
            values = getLoanerLoans(_organizationContract, _loanerAddress, privateKey)
            for i in range(len(values['_amounts'])):
                string = ''
                for key in values:
                    string += str(values[key][i]) + ' '
                loans.append(string)
    else:
        print("This account is not registered in our system.")
    return loans

In [37]:
getLoanerLoansList(loanerIndex,organizationContract,accountsConract,loaner)

Enter your password: ········


['486176 0xeCcc469d6e5288422eC9a83C6D4eb57D60aBA6d7 1590157783 13 15 ',
 '546769 0xeCcc469d6e5288422eC9a83C6D4eb57D60aBA6d7 1590157789 9 19 ']

In [ ]:
# get installment for the orgianization

In [38]:
def getLoanerInstallmentsList(_accountIndex, _loanContract, _accountsContract, _loanerAddress):
    pendingLoans = [] # 'amount loanerAddress id'
    if _accountIndex != -1:
        loanieType = _accountsContract.functions.getType(_accountIndex).call()
        if loanieType:
            privateKey = getpass("Enter your password: ")
            values = getInstallments(_loanContract, _loanerAddress, privateKey)
            for i in range(len(values['_amount'])):
                string = ''
                for key in values:
                    string += str(values[key][i]) + ' '
                pendingLoans.append(string)
    else:
        print("This account is not registered in our system.")
    return pendingLoans

In [58]:
getLoanerInstallmentsList(loanerIndex,loansContract,accountsConract,loaner)

Enter your password: ········


['37398 1592749823 1590159073 True ',
 '37398 1595341823 1590159098 True ',
 '37398 1597933823 0 False ',
 '37398 1600525823 0 False ',
 '37398 1603117823 0 False ',
 '37398 1605709823 0 False ',
 '37398 1608301823 0 False ',
 '37398 1610893823 0 False ',
 '37398 1613485823 0 False ',
 '37398 1616077823 0 False ',
 '37398 1618669823 0 False ',
 '37398 1621261823 0 False ',
 '37400 1623853823 0 False ']

In [169]:
# Confirm paid installment

In [55]:
def confirmInstallment (_loanerIndex,_loansContract,_organizationContract,_loaner,_accountsConract):
    
    installments = getLoanerInstallmentsList(_loanerIndex,_loansContract,_accountsConract,_loaner)
    print(installments)
    choice = input("enter the index of installment")
    if int(choice) >= len (installments):
        print("wrong index")
        return false
    myid = 1590157783
    result=_organizationContract.functions.confirmInstallment(int(choice),myid).transact()
    return result
    

In [57]:
confirmInstallment(loanerIndex,loansContract,organizationContract,loaner,accountsConract)

Enter your password: ········
['37398 1592749823 1590159073 True ', '37398 1595341823 0 False ', '37398 1597933823 0 False ', '37398 1600525823 0 False ', '37398 1603117823 0 False ', '37398 1605709823 0 False ', '37398 1608301823 0 False ', '37398 1610893823 0 False ', '37398 1613485823 0 False ', '37398 1616077823 0 False ', '37398 1618669823 0 False ', '37398 1621261823 0 False ', '37400 1623853823 0 False ']
enter the index of installment1


HexBytes('0x49a90049c1ccde4308aa673692d3df179fcf01ce3ac5b46b2bdac62cf3618419')